In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## nn.Linear

In [4]:
x = torch.randn(1,784)

In [6]:
x.shape

torch.Size([1, 784])

In [8]:
layer1 = nn.Linear(784, 200)
layer2 = nn.Linear(200, 200)
layer3 = nn.Linear(200, 10)

In [9]:
x=layer1(x)
x.size()

torch.Size([1, 200])

In [11]:
x = layer2(x)
x.size()

torch.Size([1, 200])

In [12]:
x = layer3(x)
x.shape

torch.Size([1, 10])

In [14]:
x = torch.randn(1,784)

In [15]:
x=layer1(x)
x=F.relu(x, inplace = True) #inplace = True是指原地进行操作，操作完成后覆盖原来的变量。
x.size()

torch.Size([1, 200])

In [16]:
x = layer2(x)
x = F.relu(x, inplace = True)
x.size()

torch.Size([1, 200])

In [17]:
x = layer3(x)
x = F.relu(x, inplace = True)
x.size()

torch.Size([1, 10])

## concisely

* inherit from nn.Module
* init layer in _init_
* implement forward()

In [20]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(784,200),
            nn.leakyReLU(inplace = True),
            nn.Linear(200,200),
            nn.leakyReLU(inplace = True),
            nn.Linear(200,10),
        )
    def forward(self, x):
        x = self.model(x)
        return x

## nn.ReLU v.s. F.relu()

* class-style API
* function-style API  
其中使用class-style，w和b都是由内部管理，而function-style则是外部管理 你可以自己定义w和b 然后对其操作

In [27]:
batch_size = 200
learning_rate = 0.01
epochs = 10

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = MLP().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1,28*28)
        data = data.to(device)
        target = target.to(device)
        logits = net(data)
        loss = criterion(logits, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        data = data.to(device)
        target = target.to(device)
        logits = net(data)
        test_loss += criterion(logits, target).item()

        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.323248
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.236568
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.213086

Test set: Average loss: 0.0008, Accuracy: 9526/10000 (95%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.174764
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.076551
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.220263

Test set: Average loss: 0.0007, Accuracy: 9565/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.102539
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.126752
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.094567

Test set: Average loss: 0.0008, Accuracy: 9582/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.093816
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.130292
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.072976

Test set: Average loss: 0.0007, Accuracy: 9627/10000 (96%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.108692
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.096645
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.102160

Test set: Average loss: 0.0008